In [2]:
import ee
ee.Authenticate()
# ee.Initialize()


Successfully saved authorization token.


In [5]:
ee.Initialize(project="ee-oderaisaack2")

In [41]:
# Define coordinates (converted from DMS to decimal)
makueni_point = ee.Geometry.Point([37.6203, -1.803])

# Create a bounding box around the point large enough to cover the county (70 km)
makueni = makueni_point.buffer(70000).bounds()

# Visual check
print("AOI type:", makueni.type().getInfo())
print("Geometry:", makueni.getInfo())

AOI type: Polygon
Geometry: {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[36.99516933021265, -2.432678449517727], [38.24736561932355, -2.432678449517727], [38.24736561932355, -1.1730793833962665], [36.99516933021265, -1.1730793833962665], [36.99516933021265, -2.432678449517727]]]}


In [175]:
def mask_l8_sr_simple(image):
    """
    Highly relaxed cloud masking (only removes high-confidence clouds)
    selecting only the 5 required bands.
    """
    qa = image.select('QA_PIXEL')

    # Mask out only the 'High Confidence' clouds
    cloud_confidence = qa.bitwiseAnd(3 << 6) 
    mask = cloud_confidence.lt(3 << 6) 
    
    # Select only the 5 bands needed for calculation (including B4 and B5 for later NDVI calculation)
    BANDS = ['SR_B2', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']
    
    # Select the bands and apply the mask
    return image.select(BANDS).updateMask(mask)

In [176]:
# maks = (ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
#       .filterBounds(makueni)
#       .map(mask_l8_sr_optimized))
landsat_collection = (ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
    .filterBounds(makueni)
    .filterDate('2013-01-01', '2025-12-31')
    .map(mask_l8_sr_simple)
    .select(['SR_B2', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7'])) 

In [ ]:
def extract_monthly_metrics(year, month):
    """
    Extract mean band values with interpolation for missing data.
    """
    start_date = ee.Date.fromYMD(year, month, 1)
    end_date = start_date.advance(1, 'month')
    
    # Filter collection for the month
    monthly_collection = landsat_collection.filterDate(start_date, end_date)
    image_count = monthly_collection.size()
    
    BAND_NAMES = ['SR_B2', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']

    # --- Robust GEE Logic: Safe Mean Image ---
    def get_safe_mean_image(collection, bands):
        """Returns the mean composite if collection is not empty, otherwise a dummy image."""
        
        constant_list = [0] * len(bands)
        dummy_image = ee.Image.constant(constant_list).rename(bands)
        
        # Conditionally return the actual mean or the dummy image.
        return ee.Algorithms.If(
            collection.size().gt(0),
            collection.mean().select(bands),
            dummy_image
        )

    mean_image = ee.Image(get_safe_mean_image(monthly_collection, BAND_NAMES))
    
    #Mean band values
    band_means = mean_image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=makueni,
        scale=30,
        bestEffort=True,
        maxPixels=1e9
    )
    
    #Interpolation Logic
    def get_band_value(band_name):
        current_value = band_means.get(band_name)
        
        def return_current():
            return current_value
        
        # Interpolation for missing data
        def interpolate_value():
            prev_month_start = start_date.advance(-1, 'month')
            prev_month_end = start_date
            next_month_start = end_date
            next_month_end = end_date.advance(1, 'month')
            
            # Get previous month's value 
            prev_collection = landsat_collection.filterDate(prev_month_start, prev_month_end)
            prev_mean_image = ee.Image(get_safe_mean_image(prev_collection, BAND_NAMES))
            prev_value = prev_mean_image.reduceRegion(reducer=ee.Reducer.mean(), geometry=makueni, scale=30, bestEffort=True, maxPixels=1e9).get(band_name)
            
            # Get next month's value 
            next_collection = landsat_collection.filterDate(next_month_start, next_month_end)
            next_mean_image = ee.Image(get_safe_mean_image(next_collection, BAND_NAMES))
            next_value = next_mean_image.reduceRegion(reducer=ee.Reducer.mean(), geometry=makueni, scale=30, bestEffort=True, maxPixels=1e9).get(band_name)
            
            # Logic for interpolation
            return ee.Algorithms.If(
                prev_value, 
                ee.Algorithms.If(
                    next_value, 
                    ee.Number(prev_value).add(ee.Number(next_value)).divide(2), 
                    prev_value    
                ),
                ee.Algorithms.If(
                    next_value,
                    next_value,
                    None 
                )
            )
        
        return ee.Algorithms.If(
            image_count.gt(0), 
            return_current(), 
            interpolate_value()
        )
        
    output_bands = {
        'B2_mean': get_band_value('SR_B2'),
        'B4_mean': get_band_value('SR_B4'),
        'B5_mean': get_band_value('SR_B5'),
        'B6_mean': get_band_value('SR_B6'),
        'B7_mean': get_band_value('SR_B7'),
    }
    
    return ee.Feature(None, {
        'year': year,
        'month': month,
        'date': start_date.format('YYYY-MM-dd'),
        'image_count': image_count,
        **output_bands,
        'interpolated_flag': ee.Algorithms.If(image_count.gt(0), 0, 1) 
    })

In [ ]:
features = []
for year in range(2013, 2026):
    for month in range(1, 13):
        features.append(extract_monthly_metrics(year, month))

In [179]:
fc = ee.FeatureCollection(features)

In [ ]:
task = ee.batch.Export.table.toDrive(
    collection=fc,
    description="Makueni_Bands_Monthly_2013_2025",
    fileFormat="CSV",
    fileNamePrefix="makueni_bands"  
)

In [181]:
task.start()
print("Export started. Check Tasks in Earth Engine.")
print("This will export monthly mean values for Landsat bands:")
print("B2 (Blue), B3 (Green), B4 (Red), B5 (NIR), B6 (SWIR1), B7 (SWIR2)")
print("You can calculate NDVI later using: (B5 - B4) / (B5 + B4)")

Export started. Check Tasks in Earth Engine.
This will export monthly mean values for Landsat bands:
B2 (Blue), B3 (Green), B4 (Red), B5 (NIR), B6 (SWIR1), B7 (SWIR2)
You can calculate NDVI later using: (B5 - B4) / (B5 + B4)
